# Overview Materi

Source: https://www.youtube.com/watch?v=LDRbO9a6XPU

Jelaskan secara singkat apa itu decision tree menurut pemahamanmu!

>Decision tree adalah struktur seperti diagram alir yang digunakan dalam machine learning untuk tugas klasifikasi dan regresi. Decision tree memecah dataset menjadi subset yang semakin kecil, sementara pada saat yang sama pohon keputusan terkait dikembangkan secara bertahap. Hasil akhirnya adalah pohon dengan node keputusan dan node daun. Node keputusan mewakili pengujian pada suatu atribut, dan setiap cabang mewakili hasil dari pengujian tersebut. Node daun mewakili label kelas (dalam klasifikasi) atau nilai numerik (dalam regresi).

# Import Data & Libraries

In [2]:
from __future__ import print_function

# label kolom
header = ["color", "diameter", "label"]

# data training
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# data testing
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# Fungsi Dasar

In [9]:
from typing import AsyncIterable
# fungsi mencari apa saja unique value dari suatu kolom
# di dalam decision tree, unique value tiap suatu kolom, misal color, diameter, dan label, perlu dicari agar bisa mengetahui informasi/data yang ada di dalam kolom dan bisa membuat pertanyaan atau aturan pada node.

# contoh penggunaan
# Dalam training_data terdapat 3 kolom, yakni color, diameter, dan label, ketika sudah dikenali apa saja unique value dari tiap kolom maka user dapat membuat split pada node, misal:
# 1. Apakah diameter buah >=2?
# 2. Apakah buah bewarna merah?
# Dari pertanyaan tsb, maka timbul percabangan untuk tiap value yang bernilai "True" dan value yang bernilai "False" sesuai dengan pertanyaan yang diberikan pada node.

Item: Green, Tipe: <class 'str'>
Item: 3, Tipe: <class 'int'>
Item: Apple, Tipe: <class 'str'>
Item: Yellow, Tipe: <class 'str'>
Item: 3, Tipe: <class 'int'>
Item: Lemon, Tipe: <class 'str'>


In [13]:
# fungsi Menghitung jumlah unique value dari suatu kolom
def class_counts(rows):
    jumlah_buah = {}
    for row in rows:
        label = row[2]
        if label not in jumlah_buah:
            jumlah_buah[label] = 0
        jumlah_buah[label] += 1
    return jumlah_buah

# contoh penggunaan
jumlah_buah = class_counts(training_data)
print(jumlah_buah)

{'Apple': 2, 'Grape': 2, 'Lemon': 1}


In [19]:
# fungsi pengecekan suatu value numerik atau bukan
def is_numeric(value):
    return isinstance(value, (float, int))

# contoh penggunaan
print(is_numeric(2025))
print(is_numeric("Three"))

True
False


In [48]:
# kelas untuk merepresentasikan pertanyaan pada decision tree
class Question:

    # inisialisasi kolom dan nilai pertanyaan
    def __init__(self, column, value):
        self.column = column
        self.value = value

    # mengecek apakah contoh data sesuai dengan pertanyaan
    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    # menampilkan pertanyaan dalam format string yang mudah dibaca
    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return f"Is {header[self.column]} {condition} {self.value}?"

# contoh penggunaan 1
# Apakah warna buahnya kuning?
q = Question(0, 'Yellow')
print(q)

example = training_data[0]
print(f"Does {example} match the question? {q.match(example)}")

example = training_data[2]
print(f"Does {example} match the question? {q.match(example)}")

# contoh penggunaan 2
# Apakah diameter buah >= 2?
q2 = Question(1, 2)
print(q2)

example = training_data[0]
print(f"Does {example} match the question? {q2.match(example)}")

example = training_data[2]
print(f"Does {example} match the question? {q2.match(example)}")

Is color == Yellow?
Does ['Green', 3, 'Apple'] match the question? False
Does ['Red', 1, 'Grape'] match the question? False
Is diameter >= 2?
Does ['Green', 3, 'Apple'] match the question? True
Does ['Red', 1, 'Grape'] match the question? False


In [53]:
# membagi dataset menjadi dua berdasarkan pertanyaan
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

# contoh penggunaan
# Pisahkan data training berdasarkan pertanyaan "Apakah warna buahnya kuning?"
true_rows, false_rows = partition(training_data, Question(0, 'Yellow'))

print("True rows (Is color == Yellow?):")
for row in true_rows:
    print(row)

print("False rows (Is color == Yellow?):")
for row in false_rows:
    print(row)

True rows (Is color == Yellow?):
['Yellow', 3, 'Apple']
['Yellow', 3, 'Lemon']
False rows (Is color == Yellow?):
['Green', 3, 'Apple']
['Red', 1, 'Grape']
['Red', 1, 'Grape']


**apa itu gini impurity?**
<br> gini impurity berfungsi mengukur tingkat ketidakmurnian atau ketidakteraturan pada sebuah simpul (node) dalam pohon

In [54]:
# menghitung nilai Gini Impurity untuk sebuah dataset
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for label in counts:
        prob_of_label = counts[label] / float(len(rows))
        impurity -= prob_of_label**2
    return impurity

# contoh penggunaan
# Hitung Gini Impurity untuk seluruh data training
print(f"Gini Impurity of training data: {gini(training_data)}")

# Hitung Gini Impurity untuk true_rows dari pertanyaan "Is color == Red?"
true_rows, false_rows = partition(training_data, Question(0, 'Red'))
print(f"Gini Impurity of true_rows (Is color == Red?): {gini(true_rows)}")

# Hitung Gini Impurity untuk false_rows dari pertanyaan "Is color == Red?"
print(f"Gini Impurity of false_rows (Is color == Red?): {gini(false_rows)}")

Gini Impurity of training data: 0.6399999999999999
Gini Impurity of true_rows (Is color == Red?): 0.0
Gini Impurity of false_rows (Is color == Red?): 0.4444444444444445


**apa itu information gain?**
<br> information gain berfungsi mengukur seberapa efektif sebuah fitur dalam memisahkan data berdasarkan kelas-kelasnya

In [47]:
# menghitung nilai Information Gain dari pemisahan dataset
def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

# contoh penggunaan
# Hitung Information Gain dari pemisahan dr pertanyaan "Is color == Yellow?"
current_uncertainty = gini(training_data)
true_rows, false_rows = partition(training_data, Question(0, 'Yellow'))
gain = info_gain(true_rows, false_rows, current_uncertainty)
print(f"Information Gain from splitting on 'Is color == Yellow?': {gain}")

# Hitung Information Gain dari pemisahan dr pertanyaan "Is diameter >= 2?"
true_rows, false_rows = partition(training_data, Question(1, 2))
current_uncertainty = gini(training_data)
gain = info_gain(true_rows, false_rows, current_uncertainty)
print(f"Information Gain from splitting on 'Is diameter >= 2?': {gain}")

Information Gain from splitting on 'Is color == Yellow?': 0.17333333333333323
Information Gain from splitting on 'Is diameter >= 2?': 0.37333333333333324


In [ ]:
# mencari pertanyaan terbaik untuk membagi dataset berdasarkan information gain tertinggi
def find_best_split(rows):
    best_gain = 0
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1

    for col in range(n_features):
        values = set([row[col] for row in rows])
        for val in values:
            question = Question(col, val)
            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the dataset
            if not true_rows or not false_rows:
                continue
            gain = info_gain(true_rows, false_rows, current_uncertainty)
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

# contoh penggunaan
best_gain, best_question = find_best_split(training_data)
print(f"Best Information Gain: {best_gain}")
print(f"Best Question: {best_question}")

# Fungsi Decision Tree

In [32]:
# merepresentasikan node daun (leaf) pada decision tree yang berisi hasil prediksi
class Leaf:

    # inisialisasi leaf dengan menghitung jumlah kemunculan tiap kelas
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [33]:
# merepresentasikan node keputusan (decision node) yang berisi pertanyaan dan cabang
class Decision_Node:

    # inisialisasi node dengan pertanyaan, cabang benar, dan cabang salah
    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [55]:
# membangun decision tree secara rekursif
def build_tree(rows):
    gain, question = find_best_split(rows)
    if gain == 0:
        return Leaf(rows)
    true_rows, false_rows = partition(rows, question)

    # Membangun true branch secara rekursif
    true_branch = build_tree(true_rows)

    # Membangun false branch secara rekursif
    false_branch = build_tree(false_rows)

    # Mengembalikan Decision_Node
    return Decision_Node(question, true_branch, false_branch)

In [56]:
# mencetak struktur decision tree secara rekursif dalam format teks
def print_tree(node, spacing=""):
    """Prints the tree recursively."""

    # base case: jika sudah mencapai leaf
    if isinstance(node, Leaf):
        print(spacing + "Predict", node.predictions)
        return

    # mencetak pertanyaan pada node saat ini
    print(spacing + str(node.question))

    # mencetak cabang true secara rekursif
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # mencetak cabang false secara rekursif
    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

# contoh penggunaan
my_tree = build_tree(training_data)
print_tree(my_tree)

Is diameter >= 3?
--> True:
  Is color == Yellow?
  --> True:
    Predict {'Apple': 1, 'Lemon': 1}
  --> False:
    Predict {'Apple': 1}
--> False:
  Predict {'Grape': 2}


In [38]:
# mengklasifikasikan satu baris data menggunakan decision tree
def classify(row, node):
    # base case: jika sudah mencapai leaf
    if isinstance(node, Leaf):
        return node.predictions

    # menentukan apakah mengikuti cabang true atau cabang false
    # dengan membandingkan nilai fitur pada baris dengan pertanyaan di node
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

# contoh penggunaan
example = training_data[0]
prediction = classify(example, my_tree)
print(f"Prediction for {example}: {prediction}")

example = training_data[1]
prediction = classify(example, my_tree)
print(f"Prediction for {example}: {prediction}")


example = training_data[2]
prediction = classify(example, my_tree)
print(f"Prediction for {example}: {prediction}")

Prediction for ['Green', 3, 'Apple']: {'Apple': 1}
Prediction for ['Yellow', 3, 'Apple']: {'Apple': 1, 'Lemon': 1}
Prediction for ['Red', 1, 'Grape']: {'Grape': 2}


In [39]:
# menampilkan prediksi pada leaf dalam format persentase
def print_leaf(counts):
    total = sum(counts.values())
    probs = {}
    for label in counts:
        probs[label] = f"{counts[label] / total * 100:.2f}%"
    return probs

# contoh penggunaan
example = training_data[0] # ['Green', 3, 'Apple']
prediction_counts = classify(example, my_tree)
prediction_probs = print_leaf(prediction_counts)
print(f"Prediction probabilities for {example}: {prediction_probs}")

example = training_data[4] # ['Yellow', 3, 'Lemon']
prediction_counts = classify(example, my_tree)
prediction_probs = print_leaf(prediction_counts)
print(f"Prediction probabilities for {example}: {prediction_probs}")

Prediction probabilities for ['Green', 3, 'Apple']: {'Apple': '100.00%'}
Prediction probabilities for ['Yellow', 3, 'Lemon']: {'Apple': '50.00%', 'Lemon': '50.00%'}


# Predict Using Decision Tree

In [41]:
# menguji decision tree dengan data uji dan membandingkan hasil prediksi dengan label asli
print("Testing the Decision Tree with testing_data:")
for row in testing_data:
    prediction_counts = classify(row, my_tree)
    prediction_probs = print_leaf(prediction_counts)
    print(f"Example: {row}, Prediction probabilities: {prediction_probs}")

Testing the Decision Tree with testing_data:
Example: ['Green', 3, 'Apple'], Prediction probabilities: {'Apple': '100.00%'}
Example: ['Yellow', 4, 'Apple'], Prediction probabilities: {'Apple': '50.00%', 'Lemon': '50.00%'}
Example: ['Red', 2, 'Grape'], Prediction probabilities: {'Grape': '100.00%'}
Example: ['Red', 1, 'Grape'], Prediction probabilities: {'Grape': '100.00%'}
Example: ['Yellow', 3, 'Lemon'], Prediction probabilities: {'Apple': '50.00%', 'Lemon': '50.00%'}
